# Scraping des données

# Méthode 1 : inspiration du cours

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import dateparser
import re

In [7]:
# Définir l'URL de base et le nombre de pages à scrapper
base_url = "https://choisirleservicepublic.gouv.fr/nos-offres/filtres/localisation/196/page/"
num_pages = 9
#169 pages en tout donc c'est le max à mettre

In [8]:
# Créer une liste vide pour stocker les annonces
annonces = []
page = 1
# Boucler sur les pages
for page in range(1, num_pages + 1):
    #Information sur l'avancé du scrapping
    print(f"Avancement du scrapping : {round((page/num_pages)*100)}%")
    # Construire l'URL complète de la page
    url = base_url + str(page)
    # Faire une requête GET pour obtenir le contenu HTML de la page
    response = requests.get(url)
    # Créer un objet BeautifulSoup pour analyser le HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    # Trouver tous les éléments qui contiennent les annonces
    elements = soup.find_all('div', class_ = 'card--offer card--offer-horizontal')
    # Boucler sur les éléments
    for element in elements:
        # Extraire le titre, le lien, la date, la référence, le type et le lieu de l'annonce
        titre = element.find('h2').get_text().strip()
        lien = element.find('h2').find('a')["href"]
        reference = lien.split('reference-')[-1].replace('/','')
        date = element.find('li', class_="fr-icon-calendar-line fr-icon--sm").get_text().strip()
        departement = element.find('li', class_= 'fr-icon-map-pin-2-line fr-icon--sm').get_text().strip()
        type = element.find('li', class_= 'fr-icon-file-line fr-icon--sm').text.split('\n')[-1].strip()

        #On nettoie les données afin qu'elle soit plus faciler à traiter par la suite (on retire les retour à la ligne, et on change le format de la date)
        titre = titre.split('\n')[-1].strip()
        departement = departement.split('\n')[1].strip()
        date = date.split('depuis le')[1].strip()
        date = dateparser.parse(date).date().isoformat()
        # Créer un dictionnaire pour stocker les informations de l'annonce
        annonce = {"titre": str(titre),"type": str(type),"reference": str(reference), "departement": str(departement),"date": str(date), "lien": str(lien),}
        # Ajouter le dictionnaire à la liste des annonces
        annonces.append(annonce)
    # Attendre un délai de 2 sec pour éviter le DDOS
    time.sleep(2)

# On crée une fonction qui retourne le département ou vide s'il n'existe pas
def get_dep_nb(texte):
    test = re.search('(\d{1,3})', texte)
    if test:
        return test.group(1)
    else:
        return ''

Avancement du scrapping : 1%
Avancement du scrapping : 1%
Avancement du scrapping : 2%
Avancement du scrapping : 2%
Avancement du scrapping : 3%
Avancement du scrapping : 4%
Avancement du scrapping : 4%
Avancement du scrapping : 5%
Avancement du scrapping : 5%
Avancement du scrapping : 6%
Avancement du scrapping : 7%
Avancement du scrapping : 7%
Avancement du scrapping : 8%
Avancement du scrapping : 8%
Avancement du scrapping : 9%
Avancement du scrapping : 9%
Avancement du scrapping : 10%
Avancement du scrapping : 11%
Avancement du scrapping : 11%
Avancement du scrapping : 12%
Avancement du scrapping : 12%
Avancement du scrapping : 13%
Avancement du scrapping : 14%
Avancement du scrapping : 14%
Avancement du scrapping : 15%
Avancement du scrapping : 15%
Avancement du scrapping : 16%
Avancement du scrapping : 17%
Avancement du scrapping : 17%
Avancement du scrapping : 18%
Avancement du scrapping : 18%
Avancement du scrapping : 19%
Avancement du scrapping : 20%
Avancement du scrapping : 

In [9]:
# Afficher le nombre d'annonces trouvées
print(f"Nombre d'annonces trouvées: {len(annonces)}")

Nombre d'annonces trouvées: 4208


In [10]:
# Transformation de la liste en dataframe
df = pd.DataFrame(annonces)
# Ajout d'une colonne numéro de département
df['departement nb'] = df['departement'].apply(get_dep_nb)

# Création d'un fichier csv qui s'exporte dans le dossier où se trouve le fichier d'execution
df.head()
df.to_excel('AnnoncesEmploisScrap.xlsx')
df.to_csv('AnnoncesEmploisScrap.csv')

# Méthode 2 : Scraping pour obtenir les catégories (A+, A, B, C) de la fonction publique pour chaque offre

Dans le cadre de cette étude, il convient d'identifier et de classer les différentes catégories pour chaque offre. À titre d'illustration de scraping, nous examinerons plus spécifiquement la catégorie A+.

In [ ]:
base_url = "https://choisirleservicepublic.gouv.fr/nos-offres/filtres/localisation/196/categorie/4327-4328/page/"
num_pages = 9

In [ ]:
# Créer une liste vide pour stocker les annonces
annonces = []
page = 1
# Boucler sur les pages
for page in range(1, num_pages + 1):
    #Information sur l'avancé du scrapping
    print(f"Avancement du scrapping : {round((page/num_pages)*100)}%")
    # Construire l'URL complète de la page
    url = base_url + str(page)
    # Faire une requête GET pour obtenir le contenu HTML de la page
    response = requests.get(url)
    # Créer un objet BeautifulSoup pour analyser le HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    # Trouver tous les éléments qui contiennent les annonces
    elements = soup.find_all('div', class_ = 'card--offer card--offer-horizontal')
    # Boucler sur les éléments
    for element in elements:
        # Extraire le titre, le lien, la date, la référence, le type et le lieu de l'annonce
        titre = element.find('h2').get_text().strip()
        lien = element.find('h2').find('a')["href"]
        reference = lien.split('reference-')[-1].replace('/','')
        date = element.find('li', class_="fr-icon-calendar-line fr-icon--sm").get_text().strip()
        departement = element.find('li', class_= 'fr-icon-map-pin-2-line fr-icon--sm').get_text().strip()
        type = element.find('li', class_= 'fr-icon-file-line fr-icon--sm').text.split('\n')[-1].strip()

        #On nettoie les données afin qu'elle soit plus faciler à traiter par la suite (on retire les retour à la ligne, et on change le format de la date)
        titre = titre.split('\n')[-1].strip()
        departement = departement.split('\n')[1].strip()
        date = date.split('depuis le')[1].strip()
        date = dateparser.parse(date).date().isoformat()
        # Créer un dictionnaire pour stocker les informations de l'annonce
        annonce = {"titre": str(titre),"type": str(type),"reference": str(reference), "departement": str(departement),"date": str(date), "lien": str(lien),}
        # Ajouter le dictionnaire à la liste des annonces
        annonces.append(annonce)
    # Attendre un délai de 2 sec pour éviter le DDOS
    time.sleep(3)

# On crée une fonction qui retourne le département ou vide s'il n'existe pas
def get_dep_nb(texte):
    test = re.search('(\d{1,3})', texte)
    if test:
        return test.group(1)
    else:
        return ''

In [ ]:
# Afficher le nombre d'annonces trouvées
print(f"Nombre d'annonces trouvées: {len(annonces)}")

In [ ]:
# Transformation de la liste en dataframe
df = pd.DataFrame(annonces)
# Ajout d'une colonne numéro de département
df['departement nb'] = df['departement'].apply(get_dep_nb)
#Catégorie A+
df['Categorie'] = 'A+'

df.to_csv('ScrapCatA+.csv')

In [26]:
df.head()

,titre,type,reference,departement,date,lien,departement nb,Categorie
0,"CAIOM Tremplin-Directeur adjoint DCL, chef du ...",Fonction publique de l'État,MINT_057ATK0003-77635,Moselle (57),2024-01-19,https://choisirleservicepublic.gouv.fr/offre-e...,57,A+
1,DIRECTEUR TERRITORIAL AUBE/HAUTE-MARNE H/F,Fonction publique de l'État,2024-1451879,Aube (10),2024-01-19,https://choisirleservicepublic.gouv.fr/offre-e...,10,A+
2,Conservateur du patrimoine (h/f) - COMMUNE DE ...,Fonction publique Territoriale,O068240101324231,Haut Rhin (68),2024-01-19,https://choisirleservicepublic.gouv.fr/offre-e...,68,A+
3,"Conseiller de recteur, adjoint à la DRAIO de l...",Fonction publique de l'État,2024-1450858,Marne (51),2024-01-18,https://choisirleservicepublic.gouv.fr/offre-e...,51,A+
4,Chargé(e) de mission lutte contre les violence...,Fonction publique de l'État,2024-1449578,Moselle (57),2024-01-18,https://choisirleservicepublic.gouv.fr/offre-e...,57,A+


Après avoir collecté les données relatives aux différentes catégories de la fonction publique, nous procéderons à leur consolidation en un fichier exhaustif.

In [37]:
df1 = pd.read_csv('S:/Desktop/Python/ScrapCatA+.csv')
df2 = pd.read_csv('S:/Desktop/Python/ScrapCatA.csv')
df3 = pd.read_csv('S:/Desktop/Python/ScrapCatB.csv')
df4 = pd.read_csv('S:/Desktop/Python/ScrapCatC.csv')


data_emploi = pd.concat([df1, df2, df3, df4])
data_emploi.to_csv('df_GrandEst.csv')

In [38]:
data_emploi.head()

,Unnamed: 0,titre,type,reference,departement,date,lien,departement nb,Categorie
0,0,"CAIOM Tremplin-Directeur adjoint DCL, chef du ...",Fonction publique de l'État,MINT_057ATK0003-77635,Moselle (57),2024-01-19,https://choisirleservicepublic.gouv.fr/offre-e...,57.0,A+
1,1,DIRECTEUR TERRITORIAL AUBE/HAUTE-MARNE H/F,Fonction publique de l'État,2024-1451879,Aube (10),2024-01-19,https://choisirleservicepublic.gouv.fr/offre-e...,10.0,A+
2,2,Conservateur du patrimoine (h/f) - COMMUNE DE ...,Fonction publique Territoriale,O068240101324231,Haut Rhin (68),2024-01-19,https://choisirleservicepublic.gouv.fr/offre-e...,68.0,A+
3,3,"Conseiller de recteur, adjoint à la DRAIO de l...",Fonction publique de l'État,2024-1450858,Marne (51),2024-01-18,https://choisirleservicepublic.gouv.fr/offre-e...,51.0,A+
4,4,Chargé(e) de mission lutte contre les violence...,Fonction publique de l'État,2024-1449578,Moselle (57),2024-01-18,https://choisirleservicepublic.gouv.fr/offre-e...,57.0,A+


# FUN : nous avons essayé la bib selenium 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

## Test 1 

In [ ]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

url = "https://choisirleservicepublic.gouv.fr/nos-offres/filtres/localisation/196/page/1/"
driver.get(url)

donnees_emploi = []

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

offres = driver.find_elements(By.XPATH, "//div[@class='card--offer card--offer-horizontal']")

for offre in offres:
    departement = offre.find_elements(By.XPATH, ".//li[@class='fr-icon-map-pin-2-line fr-icon--sm']")
    date = offre.find_elements(By.XPATH, ".//li[@class='fr-icon-calendar-line  fr-icon--sm']")
    type_fonction_publique = offre.find_elements(By.XPATH, ".//li[@class='fr-icon-file-line fr-icon--sm']")
    employeur = offre.find_elements(By.XPATH, ".//li[@class='fr-icon-user-line  fr-icon--sm']")
    secteur_emploi = offre.find_elements(By.XPATH, ".//p [@class='fr-tag default']")
    titre = offre.find_elements(By.XPATH, ".//h2/a")

    donnees.append({
        'Département': departement[0].text.strip() if departement else "NA",
        'Date': date[0].text.strip() if date else "NA",
        'Type de fonction publique': type_fonction_publique[0].text.strip() if type_fonction_publique else "NA",
        'Employeur': employeur[0].text.strip() if employeur else "NA",
        'Secteur de l\'emploi': secteur_emploi[0].text.strip() if secteur_emploi else "NA",
        'Titre': titre[0].text.strip() if titre else "NA"
        })

for d in donnees_emploi:
    print(d)

## Tentative officiel

In [ ]:
donnees_emploi = []

for page in range(1, 177): 
    url = f"https://choisirleservicepublic.gouv.fr/nos-offres/filtres/localisation/196/page/{page}/"
    driver.get(url)
    time.sleep(2)
    
    offres = driver.find_elements(By.XPATH, "//div[@class='card--offer card--offer-horizontal']")

    for offre in offres:
        departement = offre.find_elements(By.XPATH, ".//li[@class='fr-icon-map-pin-2-line fr-icon--sm']")
        date = offre.find_elements(By.XPATH, ".//li[@class='fr-icon-calendar-line  fr-icon--sm']")
        type_fonction_publique = offre.find_elements(By.XPATH, ".//li[@class='fr-icon-file-line fr-icon--sm']")
        employeur = offre.find_elements(By.XPATH, ".//li[@class='fr-icon-user-line  fr-icon--sm']")
        secteur_emploi = offre.find_elements(By.XPATH, ".//p [@class='fr-tag default']")
        titre = offre.find_elements(By.XPATH, ".//h2/a")

        donnees.append({
            'Département': departement[0].text.strip() if departement else "NA",
            'Date': date[0].text.strip() if date else "NA",
            'Type de fonction publique': type_fonction_publique[0].text.strip() if type_fonction_publique else "NA",
            'Employeur': employeur[0].text.strip() if employeur else "NA",
            'Secteur de l\'emploi': secteur_emploi[0].text.strip() if secteur_emploi else "NA",
            'Titre': titre[0].text.strip() if titre else "NA"
            })

for d in donnees_emploi:
    print(d)

In [ ]:
driver.quit()

df = pd.DataFrame(donnees_emploi)
print(df)

df1to_csv('data_emploi.csv', index=False)